In [ ]:
!pip install easyocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.9/422.9 kB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 969.6/969.6 kB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.6/286.6 kB 25.3 MB/s eta 0:00:00


In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.6/914.6 kB 9.6 MB/s eta 0:00:00


In [ ]:
import string
import easyocr

In [ ]:
reader=easyocr.Reader(['en'],gpu=False)

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

In [ ]:
dict_char_to_int={'O':'0',
                  'I':'1',
                  'J':'3',
                  'A':'4',
                  'G':'6',
                  'S':'5'}

dict_int_to_char={'0':'O',
                  '1':'I',
                  '3':'J',
                  '4':'A',
                  '6':'G',
                  '5':'S'}

In [ ]:
def write_csv(results,output_path):
    with open(output_path,'w') as f:
        f.write('{},{},{},{},{},{},{}\n'.format('frame_nmr','car_id',
                                                'car_bbox','license_plate_bbox',
                                                'license_plate_bbox_score','license_number',
                                                'license_number_score'))
        for frame_nmr in results.keys():
            for car_id in results[frame_nmr].keys():
                print(results[frame_nmr][car_id])
                if 'car' in results[frame_nmr][car_id].keys() and \
                    'license_plate' in results[frame_nmr][car_id].keys() and \
                    'text' in results[frame_nmr][car_id]['license_plate'].keys():
                    f.write('{},{},{},{},{},{},{}\n'.format(frame_nmr,
                                                            car_id,
                                                            '[{} {} {} {}]'.format(
                                                                results[frame_nmr][car_id]['car']['bbox'][0],
                                                                results[frame_nmr][car_id]['car']['bbox'][1],
                                                                results[frame_nmr][car_id]['car']['bbox'][2],
                                                                results[frame_nmr][car_id]['car']['bbox'][3]),
                                                            '[{} {} {} {}]'.format(
                                                                results[frame_nmr][car_id]['license_plate']['bbox'][0],
                                                                results[frame_nmr][car_id]['license_plate']['bbox'][1],
                                                                results[frame_nmr][car_id]['license_plate']['bbox'][2],
                                                                results[frame_nmr][car_id]['license_plate']['bbox'][3]),
                                                            results[frame_nmr][car_id]['license_plate']['bbox_score'],
                                                            results[frame_nmr][car_id]['license_plate']['text'],
                                                            results[frame_nmr][car_id]['license_plate']['text_score'])
                    )

        f.close()



In [ ]:
def license_complies_format(text):


    if len(text) != 7:
        return False

    if (text[0] in string.ascii_uppercase or text[0] in dict_int_to_char.keys()) and \
       (text[1] in string.ascii_uppercase or text[1] in dict_int_to_char.keys()) and \
       (text[2] in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'] or text[2] in dict_char_to_int.keys()) and \
       (text[3] in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'] or text[3] in dict_char_to_int.keys()) and \
       (text[4] in string.ascii_uppercase or text[4] in dict_int_to_char.keys()) and \
       (text[5] in string.ascii_uppercase or text[5] in dict_int_to_char.keys()) and \
       (text[6] in string.ascii_uppercase or text[6] in dict_int_to_char.keys()):
        return True
    else:
        return False

    # if len(text)!=10:
    #     return False
    # if (text[0] in string.ascii_uppercase or text[0] in dict_int_to_char.keys()) and \
    #    (text[1] in string.ascii_uppercase or text[0] in dict_int_to_char.keys()) and \
    #    (text[2] in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'] or text[2] in dict_char_to_int.keys()) and \
    #    (text[3] in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'] or text[3] in dict_char_to_int.keys()) and \
    #    (text[4] in string.ascii_uppercase or text[4] in dict_int_to_char.keys()) and \
    #    (text[5] in string.ascii_uppercase or text[5] in dict_int_to_char.keys()) and \
    #    (text[6] in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'] or text[2] in dict_char_to_int.keys()) and \
    #    (text[7] in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'] or text[3] in dict_char_to_int.keys()) and \
    #    (text[8] in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'] or text[2] in dict_char_to_int.keys()) and \
    #    (text[9] in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'] or text[3] in dict_char_to_int.keys()):
    #     return True
    # else:
    #     return False


In [ ]:
def format_license(text):



    license_plate_ = ''
    mapping = {0: dict_int_to_char, 1: dict_int_to_char, 4: dict_int_to_char, 5: dict_int_to_char, 6: dict_int_to_char,
               2: dict_char_to_int, 3: dict_char_to_int}
    for j in [0, 1, 2, 3, 4, 5, 6]:
        if text[j] in mapping[j].keys():
            license_plate_ += mapping[j][text[j]]
        else:
            license_plate_ += text[j]

    return license_plate_


    # license_plate_=''
    # mapping = {0: dict_int_to_char, 1: dict_int_to_char, 4: dict_int_to_char, 5: dict_int_to_char,
    #            6: dict_char_to_int, 7: dict_char_to_int,8: dict_char_to_int, 9: dict_char_to_int,
    #            2: dict_char_to_int, 3: dict_char_to_int}
    # for j in [0, 1, 2, 3, 4, 5, 6,7,8,9]:
    #     if text[j] in mapping[j].keys():
    #         license_plate_ += mapping[j][text[j]]
    #     else:
    #         license_plate_ += text[j]

    # return license_plate_

In [ ]:
def read_license_plate(license_plate_crop):
    detections=reader.readtext(license_plate_crop)
    for detection in detections:
        bbox,text,score=detection
        text=text.upper().replace(' ','')
        if license_complies_format(text):
            return format_license(text),score
    return None,None

In [ ]:
def get_car(license_plate,vehicle_track_ids):
    x1,y1,x2,y2,score,class_id=license_plate
    foundIt=False
    for track in vehicle_track_ids:
        xcar1,ycar1,xcar2,ycar2=map(int,track.to_ltrb())
        car_id=track.track_id
        if x1>xcar1 and y1>ycar1 and x2<xcar2 and y2<ycar2:
            foundIt = True
            break

    if foundIt:
        return xcar1, ycar1, xcar2, ycar2, car_id
    return -1,-1,-1,-1,-1

In [ ]:
!pip install zipfile36
import sys  # Installing the zipfile module if not already installed
if sys.version_info >= (3, 6):
    import zipfile
else:
    import zipfile36 as zipfile
import os

zip_file_path = '/content/drive/MyDrive/sort.zip'  # Update with your path
extracted_dir = '/content'  # Where to extract the files

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_dir)



In [ ]:
from ultralytics import YOLO
import cv2

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
!pip install -r /content/sort/requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: '/content/sort/requirements.txt'


In [ ]:
!pip install filterpy==1.4.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110458 sha256=6b611feebd4dc870331b7cfffb61c6937bacca187b64e6ff9c739ae94f044afb
  Stored in directory: /root/.cache/pip/wheels/12/dc/3c/e12983eac132d00f82a20c6cbe7b42ce6e96190ef8fa2d15e1
Successfully built filterpy


In [ ]:
!pip install filterpy==1.4.5


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.8/29.8 MB 24.2 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
!pip install lap==0.4.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 13.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for lap
  Running setup.py clean for lap
Failed to build lap
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (lap)


In [ ]:
from sort.sort import *

In [ ]:
# /content/sortm/sort/sort.py
# from sortm.sort.sort import *

In [ ]:
import os
if not os.path.exists("deep_sort_realtime"):
    !git clone https://github.com/levan92/deep_sort_realtime.git
    !pip install -r deep_sort_realtime/requirements.txt

Cloning into 'deep_sort_realtime'...
remote: Enumerating objects: 885, done.
remote: Counting objects: 100% (227/227), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 885 (delta 177), reused 155 (delta 155), pack-reused 658 (from 1)
Receiving objects: 100% (885/885), 16.14 MiB | 21.16 MiB/s, done.
Resolving deltas: 100% (538/538), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 73.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
  

In [ ]:
!mv deep_sort_realtime0/* .
!rm -r deep_sort_realtime0

In [ ]:
from deep_sort_realtime.deepsort_tracker import DeepSort

In [ ]:
mot_tracker = DeepSort(max_age=30, n_init=3, nn_budget=10)

In [ ]:
mot_tracker=Sort()#tracker = DeepSort(max_age=30, n_init=3, nn_budget=10)

In [ ]:
coco_model = YOLO('yolov8n.pt')

100%|██████████| 6.25M/6.25M [00:00<00:00, 30.6MB/s]


In [ ]:
license_plate_detector = YOLO('/content/license_plate_detector.pt')

In [ ]:
cap = cv2.VideoCapture('./sample1.mp4')

In [ ]:
vehicles = [2, 3, 5, 7]
frame_nmr=-1
ret=True
results={}
while ret:
    frame_nmr+=1
    ret,frame=cap.read()
    if ret:
        results[frame_nmr]={}
        detections=coco_model(frame)[0]
        detections_=[]
        for detection in detections.boxes.data.tolist():
            x1,y1,x2,y2,score,class_id=detection
            if int(class_id) in vehicles:
                detections_.append([[x1,y1,x2,y2],score])

        track_ids=mot_tracker.update_tracks(detections_,frame=frame)
        license_plates=license_plate_detector(frame)[0]
        for license_plate in license_plates.boxes.data.tolist():
            x1,y1,x2,y2,score,class_id=license_plate

            # assign license plate to car
            xcar1, ycar1, xcar2, ycar2, car_id = get_car(license_plate, track_ids)

            if car_id != -1:

                # crop license plate
                license_plate_crop = frame[int(y1):int(y2), int(x1): int(x2), :]

                # process license plate
                license_plate_crop_gray = cv2.cvtColor(license_plate_crop, cv2.COLOR_BGR2GRAY)
                _, license_plate_crop_thresh = cv2.threshold(license_plate_crop_gray, 64, 255, cv2.THRESH_BINARY_INV)

                # read license plate number
                license_plate_text, license_plate_text_score = read_license_plate(license_plate_crop_thresh)

                print(license_plate_text)

                if license_plate_text is not None:
                    results[frame_nmr][car_id] = {'car': {'bbox': [xcar1, ycar1, xcar2, ycar2]},
                                                  'license_plate': {'bbox': [x1, y1, x2, y2],
                                                                    'text': license_plate_text,
                                                                    'bbox_score': score,
                                                                    'text_score': license_plate_text_score}}
                    # print(results[frame_nmr][car_id])

# write results
write_csv(results, './test.csv')


0: 384x640 22 cars, 1 bus, 2 trucks, 161.4ms
Speed: 15.3ms preprocess, 161.4ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 license_plates, 166.9ms
Speed: 4.7ms preprocess, 166.9ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)
RN05ZZG
AP05JEO
None
AP05JEO

0: 384x640 22 cars, 2 buss, 2 trucks, 149.5ms
Speed: 7.5ms preprocess, 149.5ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 license_plates, 186.4ms
Speed: 5.4ms preprocess, 186.4ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)
KH05ZZK
None
AP05JED

0: 384x640 23 cars, 1 bus, 2 trucks, 253.8ms
Speed: 7.2ms preprocess, 253.8ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 license_plates, 254.4ms
Speed: 14.3ms preprocess, 254.4ms inference, 2.7ms postprocess per image at shape (1, 3, 384, 640)
None
AP05JEO
None

0: 384x640 24 cars, 2 buss, 1 truck, 179.9ms
Speed: 5.3ms preprocess, 179.9ms inference,

In [ ]:
print(results)

{0: {'2': {'car': {'bbox': [973, 1294, 2542, 3076]}, 'license_plate': {'bbox': [1167.167724609375, 1582.8240966796875, 1350.4793701171875, 1674.3966064453125], 'text': 'RN05ZZG', 'bbox_score': 0.6211440563201904, 'text_score': 0.20154160013474215}}, '1': {'car': {'bbox': [2078, 1425, 4840, 3416]}, 'license_plate': {'bbox': [2333.426513671875, 1776.0245361328125, 2571.674560546875, 1866.1285400390625], 'text': 'AP05JEO', 'bbox_score': 0.2855432629585266, 'text_score': 0.24347133928263426}}}, 1: {'2': {'car': {'bbox': [964, 1301, 2538, 3091]}, 'license_plate': {'bbox': [1165.97900390625, 1594.833251953125, 1349.3419189453125, 1682.2496337890625], 'text': 'KH05ZZK', 'bbox_score': 0.6286625266075134, 'text_score': 0.26265775067614316}}, '1': {'car': {'bbox': [2072, 1431, 4843, 3428]}, 'license_plate': {'bbox': [2299.319091796875, 1783.2568359375, 2511.681884765625, 1869.9228515625], 'text': 'AP05JED', 'bbox_score': 0.4938766658306122, 'text_score': 0.1831157982971341}}}, 2: {'1': {'car': {

In [ ]:
import csv
import numpy as np
from scipy.interpolate import interp1d

In [ ]:
def interpolate_bounding_boxes(data):
    # Extract necessary data columns from input data
    frame_numbers = np.array([int(row['frame_nmr']) for row in data])
    car_ids = np.array([int(float(row['car_id'])) for row in data])
    car_bboxes = np.array([list(map(float, row['car_bbox'][1:-1].split())) for row in data])
    license_plate_bboxes = np.array([list(map(float, row['license_plate_bbox'][1:-1].split())) for row in data])

    interpolated_data = []
    unique_car_ids = np.unique(car_ids)
    for car_id in unique_car_ids:

        frame_numbers_ = [p['frame_nmr'] for p in data if int(float(p['car_id'])) == int(float(car_id))]
        print(frame_numbers_, car_id)

        # Filter data for a specific car ID
        car_mask = car_ids == car_id
        car_frame_numbers = frame_numbers[car_mask]
        car_bboxes_interpolated = []
        license_plate_bboxes_interpolated = []

        first_frame_number = car_frame_numbers[0]
        last_frame_number = car_frame_numbers[-1]

        for i in range(len(car_bboxes[car_mask])):
            frame_number = car_frame_numbers[i]
            car_bbox = car_bboxes[car_mask][i]
            license_plate_bbox = license_plate_bboxes[car_mask][i]

            if i > 0:
                prev_frame_number = car_frame_numbers[i-1]
                prev_car_bbox = car_bboxes_interpolated[-1]
                prev_license_plate_bbox = license_plate_bboxes_interpolated[-1]

                if frame_number - prev_frame_number > 1:
                    # Interpolate missing frames' bounding boxes
                    frames_gap = frame_number - prev_frame_number
                    x = np.array([prev_frame_number, frame_number])
                    x_new = np.linspace(prev_frame_number, frame_number, num=frames_gap, endpoint=False)
                    interp_func = interp1d(x, np.vstack((prev_car_bbox, car_bbox)), axis=0, kind='linear')
                    interpolated_car_bboxes = interp_func(x_new)
                    interp_func = interp1d(x, np.vstack((prev_license_plate_bbox, license_plate_bbox)), axis=0, kind='linear')
                    interpolated_license_plate_bboxes = interp_func(x_new)

                    car_bboxes_interpolated.extend(interpolated_car_bboxes[1:])
                    license_plate_bboxes_interpolated.extend(interpolated_license_plate_bboxes[1:])

            car_bboxes_interpolated.append(car_bbox)
            license_plate_bboxes_interpolated.append(license_plate_bbox)

        for i in range(len(car_bboxes_interpolated)):
            frame_number = first_frame_number + i
            row = {}
            row['frame_nmr'] = str(frame_number)
            row['car_id'] = str(car_id)
            row['car_bbox'] = ' '.join(map(str, car_bboxes_interpolated[i]))
            row['license_plate_bbox'] = ' '.join(map(str, license_plate_bboxes_interpolated[i]))

            if str(frame_number) not in frame_numbers_:
                # Imputed row, set the following fields to '0'
                row['license_plate_bbox_score'] = '0'
                row['license_number'] = '0'
                row['license_number_score'] = '0'
            else:
                # Original row, retrieve values from the input data if available
                original_row = [p for p in data if int(p['frame_nmr']) == frame_number and int(float(p['car_id'])) == int(float(car_id))][0]
                row['license_plate_bbox_score'] = original_row['license_plate_bbox_score'] if 'license_plate_bbox_score' in original_row else '0'
                row['license_number'] = original_row['license_number'] if 'license_number' in original_row else '0'
                row['license_number_score'] = original_row['license_number_score'] if 'license_number_score' in original_row else '0'

            interpolated_data.append(row)

    return interpolated_data


# Load the CSV file
with open('test.csv', 'r') as file:
    reader = csv.DictReader(file)
    data = list(reader)

# Interpolate missing data
interpolated_data = interpolate_bounding_boxes(data)

# Write updated data to a new CSV file
header = ['frame_nmr', 'car_id', 'car_bbox', 'license_plate_bbox', 'license_plate_bbox_score', 'license_number', 'license_number_score']
with open('test_interpolated.csv', 'w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=header)
    writer.writeheader()
    writer.writerows(interpolated_data)

['0', '1', '2', '3', '4', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37'] 1
['0', '1', '13', '15', '16', '17', '19', '20', '21', '22', '23', '25', '27', '28', '31', '33', '34', '35', '36', '40', '41', '43', '50', '51', '52', '53', '54', '55', '56', '57'] 2
['87', '88', '91', '92', '94'] 5
['67', '68', '74', '80', '83', '84', '87', '88', '89', '90', '91', '92', '93', '94'] 8


In [ ]:
import ast

import cv2
import numpy as np
import pandas as pd

In [ ]:
def draw_border(img, top_left, bottom_right, color=(0, 255, 0), thickness=10, line_length_x=200, line_length_y=200):
    x1, y1 = top_left
    x2, y2 = bottom_right

    cv2.line(img, (x1, y1), (x1, y1 + line_length_y), color, thickness)  #-- top-left
    cv2.line(img, (x1, y1), (x1 + line_length_x, y1), color, thickness)

    cv2.line(img, (x1, y2), (x1, y2 - line_length_y), color, thickness)  #-- bottom-left
    cv2.line(img, (x1, y2), (x1 + line_length_x, y2), color, thickness)

    cv2.line(img, (x2, y1), (x2 - line_length_x, y1), color, thickness)  #-- top-right
    cv2.line(img, (x2, y1), (x2, y1 + line_length_y), color, thickness)

    cv2.line(img, (x2, y2), (x2, y2 - line_length_y), color, thickness)  #-- bottom-right
    cv2.line(img, (x2, y2), (x2 - line_length_x, y2), color, thickness)

    return img


results = pd.read_csv('./test_interpolated.csv')

# load video
video_path = 'sample1.mp4'
cap = cv2.VideoCapture(video_path)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Specify the codec
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter('./out.mp4', fourcc, fps, (width, height))

license_plate = {}
for car_id in np.unique(results['car_id']):
    results['license_number_score'] = pd.to_numeric(results['license_number_score'], errors='coerce')
    max_ = np.amax(results[results['car_id'] == car_id]['license_number_score'])
    license_plate[car_id] = {'license_crop': None,
                             'license_plate_number': results[(results['car_id'] == car_id) &
                                                             (results['license_number_score'] == max_)]['license_number'].iloc[0]}
    cap.set(cv2.CAP_PROP_POS_FRAMES, results[(results['car_id'] == car_id) &
                                             (results['license_number_score'] == max_)]['frame_nmr'].iloc[0])
    ret, frame = cap.read()

    x1, y1, x2, y2 = ast.literal_eval(results[(results['car_id'] == car_id) &
                                              (results['license_number_score'] == max_)]['license_plate_bbox'].iloc[0].replace('[ ', '[').replace('   ', ' ').replace('  ', ' ').replace(' ', ','))

    license_crop = frame[int(y1):int(y2), int(x1):int(x2), :]
    license_crop = cv2.resize(license_crop, (int((x2 - x1) * 400 / (y2 - y1)), 400))

    license_plate[car_id]['license_crop'] = license_crop


frame_nmr = -1

cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

# read frames
ret = True
while ret:
    ret, frame = cap.read()
    frame_nmr += 1
    if ret:
        df_ = results[results['frame_nmr'] == frame_nmr]
        for row_indx in range(len(df_)):
            # draw car
            car_x1, car_y1, car_x2, car_y2 = ast.literal_eval(df_.iloc[row_indx]['car_bbox'].replace('[ ', '[').replace('   ', ' ').replace('  ', ' ').replace(' ', ','))
            draw_border(frame, (int(car_x1), int(car_y1)), (int(car_x2), int(car_y2)), (0, 255, 0), 25,
                        line_length_x=200, line_length_y=200)

            # draw license plate
            x1, y1, x2, y2 = ast.literal_eval(df_.iloc[row_indx]['license_plate_bbox'].replace('[ ', '[').replace('   ', ' ').replace('  ', ' ').replace(' ', ','))
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 12)

            # crop license plate
            license_crop = license_plate[df_.iloc[row_indx]['car_id']]['license_crop']

            H, W, _ = license_crop.shape

            try:
                frame[int(car_y1) - H - 100:int(car_y1) - 100,
                      int((car_x2 + car_x1 - W) / 2):int((car_x2 + car_x1 + W) / 2), :] = license_crop

                frame[int(car_y1) - H - 400:int(car_y1) - H - 100,
                      int((car_x2 + car_x1 - W) / 2):int((car_x2 + car_x1 + W) / 2), :] = (255, 255, 255)

                (text_width, text_height), _ = cv2.getTextSize(
                    license_plate[df_.iloc[row_indx]['car_id']]['license_plate_number'],
                    cv2.FONT_HERSHEY_SIMPLEX,
                    4.3,
                    17)

                cv2.putText(frame,
                            license_plate[df_.iloc[row_indx]['car_id']]['license_plate_number'],
                            (int((car_x2 + car_x1 - text_width) / 2), int(car_y1 - H - 250 + (text_height / 2))),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            4.3,
                            (0, 0, 0),
                            17)

            except:
                pass

        out.write(frame)
        frame = cv2.resize(frame, (1280, 720))

        # cv2.imshow('frame', frame)
        # cv2.waitKey(0)

out.release()
cap.release()

In [ ]:
import pandas as pd

def clean_license_data(input_csv, output_csv):
    # Load the CSV file
    df = pd.read_csv(input_csv)

    # Remove rows where the license_number_score is 0
    df = df[df['license_number_score'] > 0]

    # Keep the highest scoring license plate for each car_id
    df = df.loc[df.groupby('car_id')['license_number_score'].idxmax()]
    df=df.drop(['car_bbox','license_plate_bbox','license_plate_bbox_score'], axis=1)

    # Save the cleaned data
    df.to_csv(output_csv, index=False)
    print(f"Cleaned data saved to {output_csv}")

# Example usage
input_csv = "test.csv"  # Change this to the actual file path
output_csv = "cleaned_test1.csv"
clean_license_data(input_csv, output_csv)


Cleaned data saved to cleaned_test1.csv


In [ ]:
from typing import final
import torch
from ultralytics import YOLO
import cv2
import time
import numpy as np
from deep_sort_realtime.deepsort_tracker import DeepSort
import os

# Clone and install DeepSORT repository if not already present
if not os.path.exists("deep_sort_realtime"):
    !git clone https://github.com/levan92/deep_sort_realtime.git
    !pip install -r deep_sort_realtime/requirements.txt

# Initialize DeepSORT tracker
tracker = DeepSort(max_age=30, n_init=3, nn_budget=10)

# Load trained gender classification model
loaded_model = torch.load('gender_classification_model.pth', map_location=torch.device('cpu') )
loaded_model.eval()

def preprocess_image(image):
    """Preprocess image before feeding to gender classification model."""
    # Resize to 224x224 and normalize if required (here, simple resize)
    image = cv2.resize(image, (224, 224))
    # Convert image to tensor and adjust dimensions (assuming model expects a tensor of shape [1,3,224,224])
    image = torch.tensor(image, dtype=torch.float32).permute(2, 0, 1).unsqueeze(0)
    # Normalize if your model was trained with normalization (add normalization here if needed)
    return image

def classify_gender(cropped_image):
    """Classify gender based on the trained model."""
    with torch.no_grad():
        image_tensor = preprocess_image(cropped_image)
        output = loaded_model(image_tensor)
        _, predicted_class = torch.max(output, 1)
        # Assuming class 0 is Male and 1 is Female
        return "Man" if predicted_class.item() == 0 else "Woman"


def detect_objects(video_path, model_path='yolov8n.pt', conf=0.5):
    """
    Detects objects in a video using YOLO and DeepSORT, computing person count,
    gender distribution, and integrating ANPR for cars.

    Args:
        video_path (str): Path to input video.
        model_path (str): Path to YOLO model.
        conf (float): Confidence threshold for detections.
    """
    # Load YOLO model
    model = YOLO(model_path)
    # Open video file
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise IOError("Error opening video stream or file")

    frame_rate = cap.get(cv2.CAP_PROP_FPS)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter('/content/detected_video2.mp4', fourcc, frame_rate, (frame_width, frame_height))

    # Dictionary to store the predicted gender for each unique track id
    track_gender = {}
    male_count = 0
    female_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        start_time = time.time()

        # Perform detection using YOLO
        results = model.predict(source=frame, conf=conf, verbose=False)

        detections = []
        for result in results:
            boxes = result.boxes.cpu().numpy()
            for box in boxes:
                b = box.xyxy[0].astype(int)
                class_id = int(box.cls[0])  # Ensure it's extracted correctly
                confidence = box.conf[0]

                # Store high-confidence detections for tracking
                if confidence > conf and model.names[class_id] in ["person", "car"]:
                    detections.append([[b[0], b[1], b[2], b[3]], confidence])

        # Update tracker with detections
        tracks = tracker.update_tracks(detections, frame=frame)

        for track in tracks:
            if not track.is_confirmed():
                continue

            track_id = track.track_id
            x1, y1, x2, y2 = map(int, track.to_ltrb())

            # Process person detections
            if model.names[class_id] == "person":
                # If this track hasn't been processed yet, classify and store its gender
                if track_id not in track_gender:
                    cropped_person = frame[y1:y2, x1:x2]
                    gender = classify_gender(cropped_person)
                    track_gender[track_id] = gender
                    if gender == "Man":
                        male_count += 1
                    elif gender == "Woman":
                        female_count += 1
                else:
                    gender = track_gender[track_id]

                # Draw bounding box and predicted gender label on frame
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, gender, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

            # Process car detections using ANPR
            elif model.names[class_id] == "car":
                cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
                df=perform_anpr(frame)
        end_time = time.time()
        fps = 1 / (end_time - start_time)
        cv2.putText(frame, f"FPS: {fps:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        out.write(frame)
    cap.release()
    out.release()
    print("Detection completed. Processed video saved.")
    print(f"Total Unique Persons Detected: {len(track_gender)}, Male: {sum(1 for g in track_gender.values() if g=='Man')}, Female: {sum(1 for g in track_gender.values() if g=='Woman')}")
def perform_anpr(frame):
    pass

# Example usage
detect_objects('/content/sample1.mp4')
